In [30]:
import sys
import pickle
import glob
import networkx as nx
import pickle
import itertools as it
import os
import shutil

import warnings
warnings.filterwarnings(action='once')

# if '/home/anna/BioSimSpace/python' not in sys.path:
#     sys.path.insert(1, '/home/anna/BioSimSpace/python')
# import BioSimSpace as BSS

# if '/home/anna/Documents/cinnabar' not in sys.path:
#     sys.path.insert(1, '/home/anna/Documents/cinnabar')
# import cinnabar

# print(cinnabar.__file__)

print("adding code to the pythonpath...")
code = '/home/anna/Documents/code/python'
if code not in sys.path:
    sys.path.insert(1, code)
import pipeline

from pipeline import *
from pipeline.analysis import *
from pipeline.prep import *
from pipeline.utils import *

# import BioSimSpace.Sandpit.Exscientia as BSS
# import BioSimSpace as BSS


BSS.__file__

import py3Dmol as _py3Dmol
from rdkit import Chem as _Chem
import tempfile as _tempfile

adding code to the pythonpath...


In [31]:
def draw_labelled_molecule(lig_0):

    # Create a temporary working directory and store the directory name.
    tmp_dir = _tempfile.TemporaryDirectory()
    work_dir = tmp_dir.name

    BSS.IO.saveMolecules(work_dir + "/molecule0", lig_0, "PDB")
    # Load the molecules into RDKit.
    rdmol0 = _Chem.MolFromPDBFile(work_dir + "/molecule0.pdb",
        sanitize=False, removeHs=False)

        
    # Set grid view properties.
    pixels = 900
    width = pixels / 2
    height = pixels

    # Create the view.
    view = _py3Dmol.view(linked=False, width=width,
            height=height)
    style = {"stick":{"colorscheme":"grayCarbon", "linewidth": 0.1}}

    # Add the molecules to the views.
    view.addModel(_Chem.MolToMolBlock(rdmol0), "mol0")

    # Set the style.
    view.setStyle({"model": 0}, style)

    # Highlight the atoms from the mapping.
    for atom0 in lig_0.getAtoms():
        p = rdmol0.GetConformer().GetAtomPosition(atom0.index())
        # p = rdmol0.GetAtomWithIdx(atom0.index())
        view.addSphere({"center" : {"x" : p.x, "y" : p.y, "z" : p.z},
                        "radius" : 0.5,
                        "color"  : "green", "alpha": 0.8},
                        )
        view.addLabel(f"{atom0.index()}",
                        {"position" : {"x" : p.x, "y" : p.y, "z" : p.z}}
                        )
        
        # Set background colour.
        view.setBackgroundColor("white")

        # Zoom to molecule.
        view.zoomTo()

    return view


def view_mapping(lig_0, lig_1):
        # Write the molecules to PDB format.
        BSS.IO.saveMolecules("molecule0", lig_0, "PDB")
        BSS.IO.saveMolecules("molecule1", lig_1, "PDB")

        # Load the molecules into RDKit.
        rdmol0 = _Chem.MolFromPDBFile("molecule0.pdb",
        sanitize=False, removeHs=False)
        rdmol1 = _Chem.MolFromPDBFile("molecule1.pdb",
        sanitize=False, removeHs=False)

        mapping = BSS.Align.matchAtoms(
                                lig_0, lig_1
                                )    
        
        # Set grid view properties.
        viewer0 = (0, 0)
        pixels = 900
        viewergrid = (2, 1)
        viewer1 = (1, 0)
        width = pixels / 2
        height = pixels

        # Create the view.
        view = _py3Dmol.view(linked=False, width=width,
                height=height, viewergrid=viewergrid)
        style = {"stick":{"colorscheme":"grayCarbon", "linewidth": 0.1}}

        # Add the molecules to the views.
        view.addModel(_Chem.MolToMolBlock(rdmol0), "mol0", viewer=viewer0)
        view.addModel(_Chem.MolToMolBlock(rdmol1), "mol1", viewer=viewer1)

        # Set the style.
        view.setStyle({"model": 0}, style, viewer=viewer0)
        view.setStyle({"model": 0}, style, viewer=viewer1)

        # Highlight the atoms from the mapping.
        for atom0, atom1 in mapping.items():
                p = rdmol0.GetConformer().GetAtomPosition(atom0)
                view.addSphere({"center" : {"x" : p.x, "y" : p.y, "z" : p.z},
                                "radius" : 0.5,
                                "color"  : "green", "alpha": 0.8},
                                viewer=viewer0)
                view.addLabel(f"{atom0} \u2192 {atom1}",
                                {"position" : {"x" : p.x, "y" : p.y, "z" : p.z}},
                                viewer=viewer0)
                p = rdmol1.GetConformer().GetAtomPosition(atom1)
                view.addSphere({"center" : {"x" : p.x, "y" : p.y, "z" : p.z},
                                "radius" : 0.5,
                                "color"  : "green", "alpha": 0.8},
                                viewer=viewer1)
                view.addLabel(f"{atom1} \u2192 {atom0}",
                                {"position" : {"x" : p.x, "y" : p.y, "z" : p.z}},
                                viewer=viewer1)
                
        # Set background colour.
        view.setBackgroundColor("white", viewer=viewer0)
        view.setBackgroundColor("white", viewer=viewer1)

        # Zoom to molecule.
        view.zoomTo(viewer=viewer0)
        view.zoomTo(viewer=viewer1)

        return view

In [57]:
lig_1 = "lig_2u"
lig_2 = "lig_2x"
prep_dir = "/home/anna/Documents/benchmark/p38_benchmark/prep"
name = "lig"

system_1 = BSS.IO.readMolecules(
    [f"{prep_dir}/{lig_1}_{name}_equil_solv.rst7", f"{prep_dir}/{lig_1}_{name}_equil_solv.prm7"])
system_2 = BSS.IO.readMolecules(
    [f"{prep_dir}/{lig_2}_{name}_equil_solv.rst7", f"{prep_dir}/{lig_2}_{name}_equil_solv.prm7"])

lig_0 = merge.extract_ligand(system_1)
lig_1 = merge.extract_ligand(system_2)
merge_molecule = merge.merge_ligands(lig_0, lig_1, **{
                                                    #   "prematch":{30:27, 34:31, 41:38, 21:18, 4:4, 2:2}, 
                                                    #   "allow ring size change":True, 
                                                    #   "allow ring breaking":True
                                                      })

In [58]:
merge.no_perturbing_atoms_average(system_1, system_2, **{
                                                    #   "prematch":{30:27, 34:31, 41:38, 21:18, 4:4, 2:2}, 
                                                    #   "allow ring size change":True, 
                                                    #   "allow ring breaking":True
                                                      })

37.5

In [59]:
BSS.Notebook.View(merge_molecule).system()

NGLWidget(gui_style='ngl')

In [59]:
# min = BSS.Protocol.Minimisation()
# min_res = run_process(merge_molecule.toSystem(), min, engine="GROMACS")

In [60]:
# BSS.Notebook.View(min_res).system()

In [60]:
view = view_mapping(lig_0, lig_1)
view

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [61]:
view = draw_labelled_molecule(lig_0)
view

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [62]:
view = draw_labelled_molecule(lig_1)
view

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [26]:
view = BSS.Align.viewMapping(lig_0, lig_1)
view

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [4]:
pert = "lig_60~lig_61"
ana_dict = analysis_protocol(file={"try pickle":False, "save pickle":False,
                                   "estimator":"MBAR",
                                   "statistical inefficiency": True,
                                   "auto equilibration": True
                                   })
ana_dict.validate()
ana_dict.print_protocol()
ana_obj = pipeline.analysis.analyse(f"/home/anna/Documents/benchmark/mcl1_benchmark/outputs/AMBER/{pert}_no_prune", analysis_protocol=ana_dict)
ana_obj._try_pickle = False
ana_obj._save_pickle = False
ana_obj.analyse_all_repeats()
# write_analysis_file(ana_obj, results_dir=f"/home/anna/Documents/benchmark/extracted/tyk2/outputs_extracted/results_decorrelated")


'file_path' must be of type 'str'.
Not recognised as file, trying to read as dictionary...
try pickle : False
save pickle : False
estimator : MBAR
statistical inefficiency : True
auto equilibration : True
kwargs : {}
method : alchemlyb
check overlap : True
truncate percentage : 0
truncate keep : end
mbar method : None
name : None
There are 3 repeats for each the bound and the free for /home/anna/Documents/benchmark/mcl1_benchmark/outputs/AMBER/lig_60~lig_61_no_prune.these are ['bound_0', 'bound_1', 'bound_2'] and ['free_0', 'free_1', 'free_2'].
MBAR free-energy analysis failed!
Unable to analyse values for 0_free, which is repeat free_0 in /home/anna/Documents/benchmark/mcl1_benchmark/outputs/AMBER/lig_60~lig_61_no_prune.
MBAR free-energy analysis failed!
Unable to analyse values for 1_free, which is repeat free_1 in /home/anna/Documents/benchmark/mcl1_benchmark/outputs/AMBER/lig_60~lig_61_no_prune.
MBAR free-energy analysis failed!
Unable to analyse values for 2_free, which is repeat 

(  nan kcal/mol,   nan kcal/mol, [])

In [12]:
BSS.FreeEnergy.Relative.analyse("/home/anna/Documents/benchmark/mcl1_benchmark/outputs/AMBER/lig_60~lig_61_pruned/free_1", **{"mbar method":"hybrid"})

AnalysisError: MBAR free-energy analysis failed with hybrid as mbar_method!

experimental
1.3386364377595559,0.2609557814854187

MBAR
current:
(1.4168 kcal/mol,
 0.0419 kcal/mol,
 [('0_repeat', 1.4175 kcal/mol, 0.0272 kcal/mol),
  ('1_repeat', 1.3592 kcal/mol, 0.0272 kcal/mol),
  ('2_repeat', 1.4737 kcal/mol, 0.0272 kcal/mol)])

decorrelate:
(1.3625 kcal/mol,
 0.0655 kcal/mol,
 [('0_repeat', 1.3991 kcal/mol, 0.0347 kcal/mol),
  ('1_repeat', 1.2604 kcal/mol, 0.0355 kcal/mol),
  ('2_repeat', 1.4280 kcal/mol, 0.0374 kcal/mol)])

decorrelate, remove burn in = False:
(1.4714 kcal/mol,
 0.0570 kcal/mol,
 [('0_repeat', 1.4429 kcal/mol, 0.0406 kcal/mol),
  ('1_repeat', 1.4201 kcal/mol, 0.0411 kcal/mol),
  ('2_repeat', 1.5513 kcal/mol, 0.0409 kcal/mol)])

both stats ineff and autoeq
(1.3770 kcal/mol,
 0.0665 kcal/mol,
 [('0_repeat', 1.4121 kcal/mol, 0.0492 kcal/mol),
  ('1_repeat', 1.2390 kcal/mol, 0.0502 kcal/mol),
  ('2_repeat', 1.4799 kcal/mol, 0.0523 kcal/mol)])

TI
current:
(1.4405 kcal/mol,
 0.0548 kcal/mol,
 [('0_repeat', 1.4191 kcal/mol, 0.0319 kcal/mol),
  ('1_repeat', 1.3871 kcal/mol, 0.0320 kcal/mol),
  ('2_repeat', 1.5152 kcal/mol, 0.0316 kcal/mol)])

decorrelate:
Average free energy of binding is 1.3515 kcal/mol and the error is 0.0906 kcal/mol for lig_ejm54~lig_ejm5

decorrelate, remove burn in = False:
Writing results. Average free energy of binding is 1.5008 kcal/mol and the error is 0.0707 kcal/mol for lig_ejm54~lig_ejm55, AMBER.

In [4]:
print(ana_obj.options_dict)

{'estimator': 'MBAR', 'method': 'alchemlyb', 'check overlap': True, 'try pickle': True, 'save pickle': True, 'auto equilibration': False, 'statistical inefficiency': False, 'truncate percentage': 0, 'truncate keep': 'end', 'mbar method': None, 'name': None}


In [2]:
lig_dict = {}

for name in ["lig_56","lig_59"]:
    lig = BSS.IO.readMolecules(f"/home/anna/Documents/benchmark/inputs/mcl1/ligands/{name}.sdf")[0]
    ligp = pipeline.prep.ligprep.lig_paramaterise(lig, ligff_query=validate.lig_ff("sage"))
    lig_dict[name] = ligp

lig0 = lig_dict["lig_56"]
lig1 = lig_dict["lig_59"]
lig0s = lig0.toSystem()
lig1s = lig1.toSystem()


In [25]:
mligs = pipeline.prep.merge.merge_system(lig0s, lig1s)
# mligs.repartitionHydrogenMass(3)

mapping, aligning and merging the ligands...


In [31]:
mligs

<BioSimSpace.System: nMolecules=1>

In [20]:
mligs_gro = mligs.copy()
protocol_min_rest, protocol_min = pipeline.prep._equilibrate.min_prots("ligprep")
min1 = pipeline.prep._equilibrate.run_process(mligs_gro, protocol_min_rest, engine="GROMACS", work_dir="/home/anna/Documents/pmx_test/gas_min/gro")
min1a = pipeline.prep._equilibrate.run_process(min1, protocol_min, engine="GROMACS", work_dir="/home/anna/Documents/pmx_test/gas_min/gro")
min1a

<BioSimSpace.System: nMolecules=1>

In [23]:
mligs_gro = mligs.copy()
protocol_min_rest, protocol_min = pipeline.prep._equilibrate.min_prots("ligprep")
min1a = pipeline.prep._equilibrate.run_process(mligs_gro, protocol_min, engine="GROMACS", work_dir="/home/anna/Documents/pmx_test/gas_min/gro")
min1a

<BioSimSpace.System: nMolecules=1>

In [29]:
mligs_somd = mligs.copy()
protocol_min_rest, protocol_min = pipeline.prep._equilibrate.min_prots("ligprep")
min2 = pipeline.prep._equilibrate.run_process(lig0s, protocol_min, engine="SOMD", work_dir="/home/anna/Documents/pmx_test/gas_min/somd")
min2


<BioSimSpace.System: nMolecules=1>

In [27]:
mligs_amb = mligs.copy()
protocol_min_rest, protocol_min = pipeline.prep._equilibrate.min_prots("ligprep")
min3 = pipeline.prep._equilibrate.run_process(mligs_gro, protocol_min_rest, engine="AMBER", work_dir="/home/anna/Documents/pmx_test/gas_min/amb")
min3a = pipeline.prep._equilibrate.run_process(min3, protocol_min, engine="AMBER", work_dir="/home/anna/Documents/pmx_test/gas_min/amb")
min3a


<BioSimSpace.System: nMolecules=1>

In [ ]:
import BioSimSpace as BSS

def extract_ligand(system):

    ligand = None
    n_residues = [mol.nResidues() for mol in system]
    n_atoms = [mol.nAtoms() for mol in system]
    for i, (n_resi, n_at) in enumerate(zip(n_residues, n_atoms)):
        if n_resi == 1 and n_at > 5:
            ligand = system.getMolecule(i)
        else:
            pass
        if ligand:
            break
    
    return ligand

def merge_ligands( ligand_0, ligand_1):

    mapping = BSS.Align.matchAtoms(
                                ligand_0, ligand_1,
                                scoring_function="rmsd_align",
                                complete_rings_only=True
                                )           
    inv_mapping = {v: k for k, v in mapping.items()}

    ligand_1_a = BSS.Align.flexAlign(ligand_1, ligand_0, inv_mapping)

    merged_ligands = BSS.Align.merge(
        ligand_0, ligand_1_a, mapping,
        allow_ring_breaking=False,
        allow_ring_size_change=False
    )

    return merged_ligands


In [ ]:

pert = "lig_27~lig_59"

lig_0 = pert.split('~')[0]
lig_1 = pert.split('~')[1]

for name in ["sys"]:

    print(f"merging for {name}...")

    # Load equilibrated inputs for both ligands
    system0 = BSS.IO.readMolecules(
        [f"sample/{lig_0}_{name}_equil_solv.rst7", f"sample/{lig_0}_{name}_equil_solv.prm7"])
    system1 = BSS.IO.readMolecules(
        [f"sample/{lig_1}_{name}_equil_solv.rst7", f"sample/{lig_1}_{name}_equil_solv.prm7"])

    lig0 = extract_ligand(system0)
    lig1 = extract_ligand(system1)

    # BSS.IO.saveMolecules(f"lig_27_{name}", lig0, ["PRM7","RST7"])
    # BSS.IO.saveMolecules(f"lig_59_{name}", lig1, ["PRM7","RST7"])

    # ligs = merge_ligands(lig0, lig1)
    # system0.removeMolecules(lig0)
    # system_final = ligs + system0

    # BSS.IO.savePerturbableSystem(f"merged_{name}", system_final)


In [ ]:
for name in ["sys"]:

    print(f"merging for {name} (extracted ligand, reloaded)...")

    # Load equilibrated inputs for both ligands
    lig0r = BSS.IO.readMolecules(
        [f"sample/{lig_0}_{name}.rst7", f"sample/{lig_0}_{name}.prm7"])[0]
    lig1r = BSS.IO.readMolecules(
        [f"sample/{lig_1}_{name}.rst7", f"sample/{lig_1}_{name}.prm7"])[0]

    ligs = merge_ligands(lig0r, lig1r)

In [ ]:

def fep_prep(pert, prot_file, num_lambda_query, engine_query, main_dir, prep_dir):

    lig_1 = pert.split('~')[0]
    lig_2 = pert.split('~')[1]

    workdir = f"{main_dir}/outputs/{engine_query}/{lig_1}~{lig_2}" # pert dir

    # parse protocol file
    print("reading in the protocol file...")
    protocol = pipeline_protocol(prot_file) # instantiate the protocol as an object
    print("validating the protocol file...")
    protocol.validate() # validate all the input
    # print("rewriting the protocol file...")
    # protocol.rewrite_protocol() # rewrite protocol file
    # add the number of lambdas and engine to the protocol
    protocol.hmr_factor(3)
    protocol.num_lambda(num_lambda_query)
    protocol.engine(engine_query)
    if protocol.name():
        workdir += f"_{protocol.name()}"

    print("the protocol is now:")
    protocol.print_protocol()

    # instantiate each system as a fepprep class with the protocol
    fepprep_obj = fepprep(protocol=protocol)

    for name, leg in zip(["lig", "sys"], ["free", "bound"]):

        # Load equilibrated inputs for both ligands
        system_1 = BSS.IO.readMolecules(
            [f"{prep_dir}/{lig_1}_{name}_equil_solv.rst7", f"{prep_dir}/{lig_1}_{name}_equil_solv.prm7"])
        system_2 = BSS.IO.readMolecules(
            [f"{prep_dir}/{lig_2}_{name}_equil_solv.rst7", f"{prep_dir}/{lig_2}_{name}_equil_solv.prm7"])

        fepprep_obj.add_system(system_1, free_bound=leg, start_end="start")
        fepprep_obj.add_system(system_2, free_bound=leg, start_end="end")

    # remove any existing files in the workdir
    try:    
        remove_tree(workdir)
    except:
        pass
    # generate folder based on fepprep protocol (both or start)
    fepprep_obj.generate_folders(workdir)

In [ ]:
prot_file = "/home/anna/Documents/benchmark/mcl1_benchmark/execution_model/protocol_gromacs5.dat"
pert = "lig_33~lig_38"
num_lambda_query = 11
engine_query = "GROMACS"
main_dir = "/home/anna/Documents/benchmark/mcl1_benchmark"
prep_dir = "/home/anna/Documents/benchmark/mcl1_benchmark/prep"

In [ ]:
for pert in ["lig_60~lig_61","lig_35~lig_49"]:
    fep_prep(pert, prot_file, num_lambda_query, engine_query, main_dir, prep_dir)


In [ ]:
transf = ["lig_60~lig_63", "lig_60~lig_61",
          "lig_35~lig_49","lig_27~lig_40",
          "lig_33~lig_38","lig_30~lig_40",
          "lig_27~lig_38","lig_56~lig_59",
          "lig_33~lig_59","lig_27~lig_59"]
engine = "GROMACS"
main_dir = "/home/anna/Documents/benchmark/mcl1_benchmark"
# methods = ["1fs", "2fs_HMR4", "4fs_HMR4", "4fs_HMR3", "2fs_HMR3", "2fs"]
methods = ["all_bonds"]

# options
ana_file = f"{main_dir}/execution_model/analysis_protocol.dat"
analysis_options = analysis_protocol(ana_file, auto_validate=True)
analysis_options.try_pickle(True)
analysis_options.print_protocol()

for method in methods:
    for trans in transf:

        # folder = path_to_dir = f"{main_dir}/outputs/{engine}/{method}_{trans}"
        # add_header_simfile(folder)
        # extraction = extract(folder)
        # # get the output from the folder to new folder
        # extraction.extract_output()
        # # get trajectory, will get rmsd by default
        # extraction.extract_frames(traj_lambdas=[], overwrite=True)

        # path_to_dir = f"{main_dir}/outputs_extracted/{engine}/{method}_{trans}"
        # final_results_folder = f"{main_dir}/outputs_extracted/results"

        # # try:
        # # using the pipeline module for analysis
        # analysed_pert = analyse(path_to_dir)
        # analysed_pert.set_options(analysis_options)
        # analysed_pert.set_options({"try pickle":False})
        # avg, error, repeats_tuple_list = analysed_pert.analyse_all_repeats()
        # print(avg, error, repeats_tuple_list)
        # # analysed_pert.plot_graphs()
        # write_analysis_file(analysed_pert, final_results_folder, method=method)
        # # except Exception as e:
        # #     print(e)
        # #     print(f"could not analyse {path_to_dir}")

        # trans = f"{trans.split('~')[1]}~{trans.split('~')[0]}" # for reverse
        # path_to_dir = f"{main_dir}/outputs_extracted/{engine}/{method}/{trans}"
        path_to_dir = f"{main_dir}/outputs_extracted/{engine}/{method}/{trans}"
        print(path_to_dir)
        final_results_folder = f"{main_dir}/outputs_extracted/results/{method}"

        try:
            # using the pipeline module for analysis
            analysed_pert = analyse(path_to_dir)
            analysed_pert.set_options(analysis_options)
            avg, error, repeats_tuple_list = analysed_pert.analyse_all_repeats()
            # analysed_pert.plot_graphs()
            write_analysis_file(analysed_pert, final_results_folder, method=method)
        except Exception as e:
            print(e)
            print(f"could not analyse {path_to_dir}")

In [ ]:
transf = ["lig_60~lig_63", "lig_60~lig_61",
          "lig_35~lig_49","lig_27~lig_40",
          "lig_33~lig_38","lig_30~lig_40",
          "lig_27~lig_38","lig_56~lig_59",
          "lig_33~lig_59","lig_27~lig_59"]
engine = "SOMD"
main_dir = "/home/anna/Documents/benchmark/extracted/mcl1"
# methods = ["1fs", "2fs_HMR4", "4fs_HMR4", "4fs_HMR3", "2fs_HMR3", "2fs"]
methods = ["hmr_prune_constrained_h-bonds"]

# options
ana_file = f"/home/anna/Documents/benchmark/mcl1_benchmark/execution_model/analysis_protocol.dat"
analysis_options = analysis_protocol(ana_file, auto_validate=True)
analysis_options.try_pickle(True)
analysis_options.print_protocol()
# analysis_options.rewrite_protocol()

for method in methods:
    for trans in transf:

        # folder = path_to_dir = f"{main_dir}/outputs/{engine}/{method}_{trans}"
        # add_header_simfile(folder)
        # extraction = extract(folder)
        # # get the output from the folder to new folder
        # extraction.extract_output()
        # # get trajectory, will get rmsd by default
        # extraction.extract_frames(traj_lambdas=[], overwrite=True)

        # path_to_dir = f"{main_dir}/outputs_extracted/{engine}/{method}_{trans}"
        # final_results_folder = f"{main_dir}/outputs_extracted/results"

        # # try:
        # # using the pipeline module for analysis
        # analysed_pert = analyse(path_to_dir)
        # analysed_pert.set_options(analysis_options)
        # analysed_pert.set_options({"try pickle":False})
        # avg, error, repeats_tuple_list = analysed_pert.analyse_all_repeats()
        # print(avg, error, repeats_tuple_list)
        # # analysed_pert.plot_graphs()
        # write_analysis_file(analysed_pert, final_results_folder, method=method)
        # # except Exception as e:
        # #     print(e)
        # #     print(f"could not analyse {path_to_dir}")

        # trans = f"{trans.split('~')[1]}~{trans.split('~')[0]}" # for reverse
        # path_to_dir = f"{main_dir}/outputs_extracted/{engine}/{method}/{trans}"
        path_to_dir = f"{main_dir}/outputs_extracted/{engine}/{trans}"
        print(path_to_dir)
        final_results_folder = f"/home/anna/Documents/benchmark/mcl1_benchmark/outputs_extracted/results/{method}"

        try:
            # using the pipeline module for analysis
            analysed_pert = analyse(path_to_dir)
            analysed_pert.set_options(analysis_options)
            avg, error, repeats_tuple_list = analysed_pert.analyse_all_repeats()
            # analysed_pert.plot_graphs()
            write_analysis_file(analysed_pert, final_results_folder, method=method)
        except Exception as e:
            print(e)
            print(f"could not analyse {path_to_dir}")